# TPM034A Machine Learning for socio-technical systems 
## `Lab session 01: Discover, explore and visualise data`

**Delft University of Technology**<br>
**Q2 2022**<br>
**Instructor:** Sander van Cranenburgh <br>
**TAs:**  Francisco Garrido Valenzuela & Lucas Spierenburg <br>

### `Instructions`

**Lab session aim to:**<br>
* Show and reinforce how models and ideas presented in class are put to practice.<br>
* Help you gather hands-on machine learning skills.<br>

**Lab sessions are:**<br>
* Learning environments where you work with Jupyter notebooks and where you can get support from TAs and fellow students.<br> 
* Not graded and do not have to be submitted. 
* A good preparation for the **assignments** (which are graded).
<br><br><br>

### `Workspace set-up`
**Option 1: Google Colab**<br>
Uncomment the following cells code lines if you are running this notebook on Colab

In [ ]:
#!git clone https://github.com/TPM34A/Q2_2022
#!pip install -r Q2_2022/requirements_colab.txt
#!mv "/content/Q2_2022/Lab_sessions/lab_session_01/data" /content/data

**Option 2: Local environment**<br>
Uncomment the following cell if you are running this notebook on your local environment. This will install all dependencies on your Python version.

In [ ]:
#!pip install -r requirements.txt

### `Application: liveability in the Netherlands` <br>
In this lab session, we will explore how liveable the Netherlands is. Liveability is high on the policy agenda. Many policies directly or indirectly aim to improve liveability, such as e.g. lowering traffic speed, curbing noise levels, maintaining public gardens and spaces, etc. But liveability is hard to study due to its elusive and complex subjective nature. In this lab session, we will discover how liveability is spatially distributed, and what factors associate positively or negatively with liveability.<br>

**Learning objectives**. After completing the following exercises you will be able to: <br>
1. Construct a data set from multiple sources using `pandas`<br>
2. Discover and visualise data, using `seaborn` and `geopandas`<br>
3. Explore associations using scatter plots and correlation heat maps<br>
4. Conduct linear regression, using `sk-learn`<br>

In [ ]:
# Import required Python packages and modules
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt


# Import selected functions and classes from Python packages
from os import getcwd
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

#### 1. Constructing a data set from multiple data sources<br>
To construct a data set from multiple data sources, we will:<br>
- A. **Load** the 3 data sets:<br>
    - A.1. Liveability data<br>
    - A.2. Population statistics data<br>
    - A.3. Topological data<br>
- B. **Clean** the data<br>
- C. **Combine** the data<br>

##### A. Loading data <br>

We first load our primary data source on **liveability**. There are three liveability data sets, with varying **spatial levels**: <br>
* `Gemeente`  Municipality 
* `Wijk`      Neighbourhood
* `Buurt`     Local neighbourhood

In [ ]:
# Get the current working directory
working_folder = getcwd()
data_folder = Path(f'data')
print(data_folder)

In [ ]:
# Load liveability data, using pandas
gemeente_liv_data = pd.read_csv(data_folder/'gemeente_liveability.csv')
wijk_liv_data = pd.read_csv(data_folder/'wijk_liveability.csv')
buurt_liv_data = pd.read_csv(data_folder/'buurt_liveability.csv')
print(list(gemeente_liv_data.keys()))

*Description of variables*<br>
`gm_code`&nbsp;&nbsp;&nbsp;Unique code for gemeente <br>
`versie `&nbsp;&nbsp;&nbsp;Version of the 'Leefbaarometer model' used <br>
`jaar   `&nbsp;&nbsp;&nbsp;&nbsp;&emsp;Year <br>
`gm_naam`&emsp;Name of gemeente <br>

*Liveability indicators:* <br>
`lbm`   &emsp;**lbm** is the **key variable of interest** in this lab session. It is composed of the variables below: <br>
`afw`   &emsp;Deviation from previous model <br>
`fys`   &emsp;Physical environment<br>
`onv`   &emsp;Nuisance and insecurity<br>
`soc`   &emsp;Social cohesion<br>
`vrz`   &emsp;Amenities<br>
`won`   &emsp;Housing stock<br>

We load the second source of data. These data contains population statistics.

In [ ]:
# Load files on population statistics
gemeente_pop_data = pd.read_csv(data_folder/'gemeente_2020_pop.csv', sep=';')
wijk_pop_data = pd.read_csv(data_folder/'wijk_2020_pop.csv', sep=';')
buurt_pop_data = pd.read_csv(data_folder/'buurt_2020_pop.csv', sep=';')
gemeente_pop_data.keys()

Finally, we load data on the topology of gemeenten, wijken and buurten. To do this, load the **shape files** using geopandas.

In [ ]:
# Load shape files
gemeente_shape = gpd.read_file(data_folder/'gemeente/gemeente 2020.shp')
wijk_shape = gpd.read_file(data_folder/'wijk/wijk 2020.shp')
buurt_shape = gpd.read_file(data_folder/'buurt/buurt 2020.shp')
buurt_shape.info()

##### B. Cleaning data
First, we clean the liveability data sets. <br>
More specifically, we: <br>
* Extract only those data points for which we have a liveability score.
* Extract only those data points that are created by `Leefbaarometer 3.0` and for year `2020`.

In [ ]:
def clean_liv_data(liv_data):

    # Remove rows where lbm is NaN
    liv_data.dropna(subset = ['lbm'], inplace = True)

    # Extract subsets: only year 2020 and Leefbaarometer 3.0 data
    liv_data = liv_data.loc[(liv_data.versie ==  'Leefbaarometer 3.0') & (liv_data.jaar == 2020)]

    return liv_data

gemeente_liv_data = clean_liv_data(gemeente_liv_data)
wijk_liv_data = clean_liv_data(wijk_liv_data)
buurt_liv_data = clean_liv_data(buurt_liv_data)

Next, we clean the population data sets.<br>
More specifically, we: <br>
* Remove useless data points (i.e. water areas, opposed to land areas)
* Drop data points where the percentage of owner occupied homes and rented homes equals 0 (for later analyses)

In [ ]:
def clean_pop_data(pop_data):

    # Extract subsets: only data where is land
    pop_data  = pop_data.loc[pop_data.H2O == 'NEE'] 

    # Remove rows where P_KOOPWON == 0 or P_HUURWON == 0
    pop_data = pop_data.loc[(pop_data.P_KOOPWON>0) | (pop_data.P_HUURWON>0)]

    return pop_data

gemeente_pop_data = clean_pop_data(gemeente_pop_data)
wijk_pop_data = clean_pop_data(wijk_pop_data)
buurt_pop_data = clean_pop_data(buurt_pop_data)

##### C. Combining data
Now, we pull together the population and liveability data sets, using pandas' `merge` function. We use `GM_CODE`, `WK_CODE` and `BU_CODE` as key indices to merge on.<br>
For the `merge` operation, the keys must be the same. Therefore, we first capitalise the keys of the shape files.<br>

In [ ]:
# Capitalise column name of variables to merge on (to be the same across data sets)
gemeente_liv_data.rename(columns={"gm_code": "GM_CODE"}, inplace=True)
wijk_liv_data.rename(columns={"wk_code": "WK_CODE"}, inplace=True)
buurt_liv_data.rename(columns={"bu_code": "BU_CODE"}, inplace=True)

# Merge DataFrames
gemeente_data = gemeente_pop_data.merge(gemeente_liv_data, on ="GM_CODE", how = "inner")
wijk_data = wijk_pop_data.merge(wijk_liv_data, on="WK_CODE", how = "inner")
buurt_data = buurt_pop_data.merge(buurt_liv_data, on="BU_CODE", how = "inner")

Next, we add the topological (shape) data, using `merge`. Again, we use `GM_CODE`, `WK_CODE`, and `BU_CODE` as indices.<br>
For the `merge` operation, the keys must be the same. Therefore, we first capitalise the keys of the shape files.<br>

In [ ]:
# Capitalise column name of key variables
gemeente_shape = gemeente_shape.rename(columns={"gm_code": "GM_CODE"}).drop(columns = 'gm_naam')
wijk_shape = wijk_shape.rename(columns={"wk_code": "WK_CODE"}).drop(columns = 'wk_naam')
buurt_shape = buurt_shape.rename(columns={"bu_code": "BU_CODE"}).drop(columns = 'bu_naam')

# Merge the GeoDataFrame and the DataFrame
gemeente_df = gemeente_shape.merge(gemeente_data, on="GM_CODE")
wijk_df = wijk_shape.merge(wijk_data, on="WK_CODE")
buurt_df = buurt_shape.merge(buurt_data, on="BU_CODE")

### <span style="color:skyblue">Exercise 1: Exploring the data</span> 

`A` Load data file `Gemeenten alfabetisch 2020.excel` into a pandas DataFrame, using `pd.read_excel()`.<br>

`B` Explore the content, e.g. using `.describe()`, `.head()`, and `.info()`
<br>

`C` Add the provincies of the Netherlands to the `gemeente_df` DataFrame, the `wijk_df` DataFrame. and the `buurt_df` DataFrame.<br>
To do so, you need to merge two data set, based on the `GM_CODE`

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

#### 2. Discovering and visualising the data
After having constructed our data set, it is time to explore the data more in depth. To do so, we use pandas `describe`, `head` and `info` functions. Additionally, to visualise the data, we create histograms (empirical cumulative density function plots). This is done using **seaborn** package. 

Let's first look at the key variable of interest: `lbm` (liveability), and visualise its ditribution using a histogram and a cumulative density function plot. 

In [ ]:
# Create histogram and empirical CDF for variable(s) of interest
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharex=True)
sns.histplot(ax = axes[0],x = wijk_df.lbm)
ecdf_data = sns.ecdfplot(ax = axes[1],x = wijk_df.lbm)
axes[0].set_xlabel("lbm score")
axes[1].set_xlabel("lbm score")
axes[1].grid(True,linewidth = 0.5)
axes[1].minorticks_on()
axes[1].grid(which='minor', linestyle=':', linewidth='0.5', color='black')


**interpretation of results**
* Liveability of wijken is approximately normally distributed (no skew / higher moments)
* The mean and median liveability of wijken is ~4.15
* The domain of liveability is ~ [3.7-4.6]

### <span style="color:skyblue">Exercise 2: Distribution of social cohesion</span> 
Besides liveability, in the liveability data there is also a variable on social cohesion `soc`.<br>
`A` Conduct several descriptive analyses, such as histograms, boxplots, etc, on the variable `soc`<br>
`B` Interpret your results

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

An important step when discovering data is to evaluate their **face validity**. To do so, let's see whether to top-rated wijken seems to make sense.

In [ ]:
# List top-5 best wijken, based on liveability
wijk_df.nlargest(5,'lbm').head(5)

### <span style="color:skyblue">Exercise 3: Face validity of worst neighbourhoods (wijken)</span> 
The best scoring neighbourhoods seem to make sense. Let's also look at the worst scoring neighbourhoods.<br>
`A` List the 10 neighourhoods (wijken) which score lowest on `lbm`

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

As we have added topological data to the dataframe, we can visualise the data in space. <br> 
To further discover the data, let's first look at the municipality level. We color municipalities based on the average liveability score.

In [ ]:
# Plot the liveability in the Netherlands at the wijk-level
fig, ax = plt.subplots(figsize=(60,60))
wijk_df.plot(ax=ax, column = 'lbm', legend = True)
gemeente_df.plot(ax=ax, color  = 'none', edgecolor='black')
_=gemeente_df.apply(lambda x: ax.annotate(text=x['GM_NAAM'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1)
# gemeente_df.apply(lambda x: '' if x['AANT_INW']<50000 else ax.annotate(text=x['GM_NAAM'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1)

**Interpretation of results**
* Wijken located near the coast (including the islands) score above average.
* Larger cities (e.g. AMS, RT, Tilburg, Utrecht, The Hague) seem to have multiple very poor performing wijken
* In some municipalities, data are patchy (missing)

Next, we do the same, but at the lowest spatial level (buurten) focusing on the municipality of Delft. 

In [ ]:
# Plot the liveability in Delft at the buurt-level
fig, ax = plt.subplots(figsize=(40,40))
buurt_df.loc[buurt_df.GM_NAAM == 'Delft'].plot(ax=ax, column = 'lbm', legend = True)
gemeente_df.loc[gemeente_df.GM_NAAM == 'Delft'].plot(ax=ax, color  = 'none', edgecolor='black')
buurt_df.loc[buurt_df.GM_NAAM == 'Delft'].apply(lambda x: ax.annotate(text=x['BU_NAAM'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1)
plt.show()

### <span style="color:skyblue">Exercise 4: Visualise local neighbourhoods in Zuid-Holland</span> 
`A` Plot the liveability score in Zuid-Holland at the buurt level

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

#### 3. Exploring associations
To explore associations, we use correlation heat maps, scatter plots and boxplots <br>

In [ ]:
# heatmap of correlations
# Create plot
fig, axes = plt.subplots(figsize=(10, 10))
fig.set_tight_layout(True)

# Compute correlation matrix
corr = wijk_df[['BEV_DICHTH','AANT_INW','AF_SUPERM','AF_RESTAU','P_KOOPWON','P_HUURWON', 'fys', 'onv', 'soc', 'vrz', 'won','lbm']].corr()

# Create upper triangular matrix to mask the upper triangular part of the heatmap
corr_mask = np.triu(np.ones_like(corr, dtype=bool))

# Generate a custom diverging colormap (because it looks better)
corr_cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr, mask = corr_mask, cmap=corr_cmap, annot=True,square = True, linewidths=.5, ax = axes)
plt.show()

**Interpretation of the results** <br>
1. Pearson corr coefficients range from [-0.99,0.86] 
2. *P_HUURWON* and *P_KOOPWON* correlate (almost) 1 to 1 (which makes sense).<br>

Looking at the bottom row for lbm, we see that:<br>

3. *BEV_DICHTH*, *AANT_INW*, *P_HUURWON* correlate negatively with liveability
4. *AF_SUPERM*, *AF_RESTAU*, *P_KOOPWON*, *fys*, *onv*, *soc*, *vrz*, *won* correlate positively with liveability
5. lbm correlates strongest with *onv* (nuisance and insecurity), *soc* (social cohesion), *won* (housing stock)


In [ ]:
# Further explore linear associations using scatter plots
# plot
fig, axes = plt.subplots(3, 2, figsize=(20, 10))
fig.set_tight_layout(True)
sns.scatterplot(ax = axes[0,0],x = wijk_df.BEV_DICHTH, y = wijk_df.lbm, alpha = 0.2)
sns.scatterplot(ax = axes[0,1],x = wijk_df.AANT_INW, y = wijk_df.lbm, alpha = 0.2)   
sns.scatterplot(ax = axes[1,0],x = wijk_df.AF_SUPERM, y = wijk_df.lbm, alpha = 0.2)
sns.scatterplot(ax = axes[1,1],x = wijk_df.AF_RESTAU, y = wijk_df.lbm, alpha = 0.2)
sns.scatterplot(ax = axes[2,0],x = wijk_df.P_KOOPWON[wijk_df.P_KOOPWON>0], y = wijk_df.lbm[wijk_df.P_KOOPWON>0], alpha = 0.2)   
sns.scatterplot(ax = axes[2,1],x = wijk_df.P_HUURWON[wijk_df.P_HUURWON>0], y = wijk_df.lbm[wijk_df.P_HUURWON>0], alpha = 0.2)   


axes[0,0].set_xlabel('Population density [#/km^2]')
axes[0,1].set_xlabel('Population size [#]')
axes[1,0].set_xlabel('Distance to supermarket [km]')
axes[1,1].set_xlabel('Distance to restaurant [km]')
axes[2,0].set_xlabel('Percentage owner occupied homes [%]')
axes[2,1].set_xlabel('Percentage rental homes [%]')

**Interpretation of the results** <br>
* The scatter plots show the bivariate relations with liveability is strongest for *P_KOOPWON* and *P_HUURWON*.
* For *BEV_DICHTH*, *AANT_INW*, *AF_RESTAU* and *P_KOOPWON* the data seems skewed, condensed around x = 0. 

**Boxplot analysis** <br>

When one of the features is categorical a **boxplot** can be insightful. <br>
Boxplots are informative as they also reveal insights on the locality, spread and skewness of numerical data through their quartiles, see https://en.wikipedia.org/wiki/Box_plot.<br>
Below we create a boxplot showing the liveability across a selected number of municipalities based on the liveabilities scores at the buurt level.
Add your own municipality to see if the liveability scores are in line with your own perceptions.

In [ ]:
# Box plot
fig, axes = plt.subplots(1, 1, figsize=(15, 10), sharex=True)
GM_select = ['Amsterdam','Rotterdam','Utrecht',"'s-Gravenhage",'Groningen','Tilburg']
df_GM_select = buurt_df[buurt_df['GM_NAAM'].isin(GM_select)]
sns.boxplot(ax = axes, y = df_GM_select.lbm, x = df_GM_select.GM_NAAM, orient="v" )

axes.grid(True,linewidth = 0.5)
plt.show()

**Interpretation of results**
* Looking at the **median** scores, Amsterdam and Groninger are the best municipalities to live in. <br>
* Unlike Groningen, Amsterdam also has very unliveable buurten. This can be seen from the outliers at the lower side.
* Larger cities seem to have a higher variance in the liveability scores across buurten (which is to be expected).

<br>

### <span style="color:skyblue">Exercise 5:  Scatter the median liveability scores of gemeentes based on wijk and buurt level data</span> 
`A` Create a scatter plot in which the median liveability score of the gemeente as computed using wijk level data is at the x-axis and the median liveability score as computed using buurt level data is at the y-axis. (hint: use the pandas method groupby)<br>
`B` Interpret the results


In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

## 4. Conducting multiple linear regression analyses
Regression models are often used to obtain first insights on the relationship between a (scalar) target and one or more features. Regression models come in many different forms: e.g. simple, multiple, ordinal, Poisson, etc. Usually, a multiple linear regression model serves as a good benchmark.

In ML the focus is usually on generalisation. That is, the model performance is evaluated by looking at the performance on an unseen set of the data (generalisation performance).<br>

For these analyses, we use the **sk-learn** Python library. This Python library is especially designed machine learning.
Having seen the scatter plots and bar plots, we focus on the following 7 features: *ANNT_INW*, *BEV_DICHTH*, *P_KOOPWON*, *fys*, *soc*, *won*, and *onv*

In [ ]:
# Set the seed number for reproducibility. It governs which data points go to the test set and which go to the training set
rs = 12345

# Create X
X_lin = pd.DataFrame([buurt_df["AANT_INW"].div(100000),buurt_df["BEV_DICHTH"].div(10000), buurt_df["P_KOOPWON"].div(100),buurt_df["fys"],buurt_df["soc"],buurt_df["won"],buurt_df["onv"]]).transpose()


# Create Y
Y = buurt_df.lbm

# Create linear regression object
regr = LinearRegression(fit_intercept = True)

# Split the data in a train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X_lin, Y, test_size=0.25,random_state = rs)

# Fit the model on the training data
regr.fit(X_train,Y_train)

# Evaluate the model generalisation performance on the Train and Test data sets
Y_pred_train = regr.predict(X_train)
mse_train = mean_squared_error(Y_train,Y_pred_train)
R2_train = r2_score(Y_train,Y_pred_train)

Y_pred_test = regr.predict(X_test)
mse_test  = mean_squared_error(Y_test, Y_pred_test)
R2_test = r2_score(Y_test,Y_pred_test)

# Print results
print('Results linear multiple regression model')
print(f'Mean Squared Error Train | Test: \t{mse_train:>7.4f}\t|  {mse_test:>7.4f}')
print(f'R2                 Train | Test: \t{R2_train:>7.4f}\t|  {R2_test:>7.4f}\n')
print('Weights')
print(f'Intercept: \t\t\t {regr.intercept_:>7.4f}')
for n in range(len(regr.coef_)):
    print(f'Weight_{X_lin.keys()[n]:10s} \t\t {regr.coef_[n]:>7.4f}')

**Interpretation of results**
* Looking at the MSE and R2, we see that the performance on the **test** data set is roughly equal to the performance on the **train** data set. This tells us that the model is not overfitting.
* An advantage of regression models is that their weights are interpretable. For instance, the positive and comparatively large weight associated with *BEV_DICHTH* tells us that high population density is associated with high liveability. 
<br>

### <span style="color:skyblue">Exercise 6:  Impact of seed number on the regression results</span> 
`A` Re-do the regression using different seed numbers (e.g. 1,2,3,4. etc) and reflect on the stability of the results<br>

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)

### <span style="color:skyblue">Exercise 7:  Generalisation</span> 
`A` Train a new extended multiple regression model. To do so, add 13 extra features to the model that could be associated with liveability. For this extended model, compute the Mean Squared Error and R2 both for the train and test data sets.<br>
`B` The picture below shows the Bias-Variance trade-off. Looking at the MSE and R2 of the model with 7 features and the extended model (with 20 features), where do you think these two models can be placed in the picture (Region A, B, C, D, or E)?  Explain your answer.
<br><br><br>
![Bias_variance_tradeoff](data/Bias_variance.png)

In [ ]:
# CODE YOUR ANSWERS HERE (Use as many cells as you need)